In [1]:
import pandas as pd
from glob import glob
import gsw
from geopy.distance import distance
import numpy as np
import seawater as sw

def zmld_boyer(s, t, p):
    """
    https://github.com/pyoceans/python-oceans/blob/master/oceans/sw_extras/sw_extras.py
    Computes mixed layer depth, based on de Boyer Montégut et al., 2004.
    Parameters
    ----------
    s : array_like
        salinity [psu (PSS-78)]
    t : array_like
        temperature [℃ (ITS-90)]
    p : array_like
        pressure [db].
    Notes
    -----
    Based on density with fixed threshold criteria
    de Boyer Montégut et al., 2004. Mixed layer depth over the global ocean:
        An examination of profile data and a profile-based climatology.
        doi:10.1029/2004JC002378
    dataset for test and more explanation can be found at:
    http://www.ifremer.fr/cerweb/deboyer/mld/Surface_Mixed_Layer_Depth.php
    Codes based on : http://mixedlayer.ucsd.edu/
    """
    m = len(np.nonzero(~np.isnan(s))[0])

    if m <= 1:
        mldepthdens_mldindex = 0
        mldepthptemp_mldindex = 0
        return mldepthdens_mldindex, mldepthptemp_mldindex
    else:
        # starti = min(find((pres-10).^2==min((pres-10).^2)));
        starti = np.min(
            np.where(((p - 10.0) ** 2 == np.min((p - 10.0) ** 2)))[0]
        )
        starti = 0
        pres = p[starti:m]
        sal = s[starti:m]
        temp = t[starti:m]

        pden = sw.dens0(sal, temp) - 1000

        mldepthdens_mldindex = m - 1
        for i, pp in enumerate(pden):
            if np.abs(pden[starti] - pp) > 0.03:
                mldepthdens_mldindex = i
                break

        # Interpolate to exactly match the potential density threshold.
        presseg = [pres[mldepthdens_mldindex - 1], pres[mldepthdens_mldindex]]
        pdenseg = [
            pden[starti] - pden[mldepthdens_mldindex - 1],
            pden[starti] - pden[mldepthdens_mldindex],
        ]
        P = np.polyfit(presseg, pdenseg, 1)
        presinterp = np.linspace(presseg[0], presseg[1], 3)
        pdenthreshold = np.polyval(P, presinterp)

        # The potential density threshold MLD value:
        ix = np.max(np.where(np.abs(pdenthreshold) < 0.03)[0])
        mldepthdens_mldindex = presinterp[ix]

        # Search for the first level that exceeds the temperature threshold.
        mldepthptmp_mldindex = m - 1
        for i, tt in enumerate(temp):
            if np.abs(temp[starti] - tt) > 0.2:
                mldepthptmp_mldindex = i
                break

        # Interpolate to exactly match the temperature threshold.
        presseg = [pres[mldepthptmp_mldindex - 1], pres[mldepthptmp_mldindex]]
        tempseg = [
            temp[starti] - temp[mldepthptmp_mldindex - 1],
            temp[starti] - temp[mldepthptmp_mldindex],
        ]
        P = np.polyfit(presseg, tempseg, 1)
        presinterp = np.linspace(presseg[0], presseg[1], 3)
        tempthreshold = np.polyval(P, presinterp)

        # The temperature threshold MLD value:
        ix = np.max(np.where(np.abs(tempthreshold) < 0.2)[0])
        mldepthptemp_mldindex = presinterp[ix]

        return mldepthdens_mldindex, mldepthptemp_mldindex
    
dfCTD = pd.concat([pd.read_csv(file) for file in glob('CTD_os*.csv')])
dfCTD = dfCTD[dfCTD['profile_id'].notna()]
dfCTD = dfCTD.drop(labels=['S_42'],axis=1)
dfCTD = dfCTD.astype({'pressure': float, 'latitude': float, 'longitude': float, 'S_41': float, 'T2_35': float,'T_28': float})
dfCTD['time'] = pd.to_datetime(dfCTD.time)
dfCTD['depth'] = -1*gsw.z_from_p(dfCTD.pressure, dfCTD.latitude)
dfCTD = dfCTD.sort_values(by='time')
dfCTD2017 = dfCTD[(dfCTD.time.dt.year == 2017) & (dfCTD.time > pd.to_datetime('8-01-2017').tz_localize ('UTC'))]
dfCTD2019 = dfCTD[(dfCTD.time.dt.year == 2019) & (dfCTD.time > pd.to_datetime('8-01-2019').tz_localize ('UTC'))]
dfCTD_clean = pd.concat([dfCTD2017,dfCTD2019])
dfCTD_clean = dfCTD_clean.rename(columns={'S_41':'s','T_28':'t'}).drop(columns=['T2_35'])

dfEvents = pd.read_csv('../catchData/2017_2019/AIERP_EventData.csv')
dfEvents = dfEvents[['SURVEY','EVENT_ID','EQ_TIME','EQ_LONGITUDE','EQ_LATITUDE']]
dfEvents['EQ_TIME'] = pd.to_datetime(dfEvents.EQ_TIME,format='%d-%b-%y %H.%M.%S.%f %p')
dfEvents = dfEvents[dfEvents.SURVEY <1000000]

In [55]:
def ctdStats(df):
    dfStations = df[['profile_id','latitude','longitude','time']].drop_duplicates()
    dists = []
    for year in df.time.dt.year.unique():
        dfCur = dfStations[dfStations.time.dt.year == year]
        nPoints = len(dfCur.latitude)
        for i in range(nPoints):
            curDists = []
            for ii in range(nPoints):
                if ii == i:
                    continue
                curDists.append(distance((dfCur.latitude.values[i],dfCur.longitude.values[i]),(dfCur.latitude.values[ii],dfCur.longitude.values[ii])).nm)
            dists.append(np.min(curDists))
    dfStations['Radius'] = np.array(dists)/2
    S_max, S_min, S_surf, S_bot, S_mean, T_min, T_max, T_surf, T_bot, T_mean,mld,Depth_max = [[] for i in range(12)]

    for pid in df.profile_id.unique():
        dfCur = df[(df.profile_id==pid)].sort_values('pressure')
        S_max.append(dfCur.s.max())
        S_min.append(dfCur.s.min())
        S_surf.append(dfCur[dfCur.depth <= 5].s.mean())
        S_bot.append(dfCur[dfCur.depth > dfCur.depth.max()-5].s.mean())
        S_mean.append(dfCur.s.mean())
        T_max.append(dfCur.t.max())
        T_min.append(dfCur.t.min())
        T_surf.append(dfCur[dfCur.depth <= 5].t.mean())
        T_bot.append(dfCur[dfCur.depth > dfCur.depth.max()-5].t.mean())
        T_mean.append(dfCur.t.mean())
        Depth_max.append(dfCur.depth.max())
        try:
            mld1, mld2 = zmld_boyer(dfCur.dropna().s.values, dfCur.dropna().t.values, dfCur.dropna().pressure.values)
        except:
            print(pid)
            mld2 = np.nan
        mld.append(mld2)
    dfStats = pd.DataFrame({'profile_id':df.profile_id.unique(),'Depth_max':Depth_max,'S_max':S_max, 'S_min':S_min, 'S_surf':S_surf, 'S_bot':S_bot,'S_mean':S_mean, 'T_min':T_min, 'T_max':T_max, 'T_surf':T_surf, 'T_bot':T_bot,'T_mean':T_mean,'MLD':mld})
    return dfStations.merge(dfStats)

def matchTrawl(dfEvents, dfCTDStats):
    profileList,distList,timeList = [],[],[]
    for year in [2017,2019]:
        dfTrawls = dfEvents[dfEvents.SURVEY == (year*100)+1]
        dfEnvCur = dfCTDStats[dfCTDStats.time.dt.year == year]
        for i in range(len(dfTrawls)):
            curTrawl = (dfTrawls.EQ_LATITUDE.values[i],dfTrawls.EQ_LONGITUDE.values[i])#(dfEnvCur.latitude.values[i],dfEnvCur.longitude.values[i])        
            curDist = 99999
            for ii in range(len(dfEnvCur)):
                dist = distance(curTrawl,(dfEnvCur.latitude.values[ii],dfEnvCur.longitude.values[ii])).km
                if dist < curDist:
                    if abs((dfEnvCur.time.values[ii]-dfTrawls.EQ_TIME.values[i]).astype('timedelta64[D]')).astype('int') < 2:
                        curDist = dist
                        curProfile = dfEnvCur.profile_id.values[ii]
                        curTime = dfEnvCur.time.values[ii]
            profileList.append(curProfile)
            distList.append(curDist)
            timeList.append(curTime)
    dfEvents['profile_id'] = profileList
    dfEvents['CTD_DISTANCE'] = distList
    dfTrawlCTD = pd.merge(dfEvents,dfCTDStats).drop(columns='Radius')
    dfTrawlCTD = dfTrawlCTD.rename(columns={'profile_id':'CTD_PROFILE','latitude':'CTD_LATITUDE','longitude':'CTD_LONGITUDE','time':'CTD_TIME'})
    return dfTrawlCTD

In [56]:
#dfCTDStats = ctdStats(dfCTD_clean)
dfTrawlCTD = matchTrawl(dfEvents,dfCTDStats)
dfTrawlCTD.to_csv('Chukchi17_19_trawl_ctd.csv',index=False)

## Additional requests for Baker et al

In [54]:
# Add classifiers
# 1 = meltwater
# 2 = BCWW <32.4 sal
# 3 = BCWW > 32.4
# 4 ?
# 5 = BCSW
# 6 = ACW
# 7 (new) AtlW

def wmClass(temp,sal):
    if (temp < 7) & (sal < 30):
        wm = 1 # meltwater
    elif (temp < 0) & (sal < 32.4) & (sal > 30):
        wm = 2 # BCWW <32.4 sal
    elif (temp < 0) & (sal > 32.4) & (sal < 33.5):
        wm = 3 # BCWW > 32.4 sal
    elif (temp > 0) & (temp < 7) &(sal > 30) & (sal < 33.5):
        wm = 5 # BCSW
    elif (temp > 7):
        wm = 6 # ACW
    elif  (temp < 1) &(sal > 33.5):
        wm = 7 # AtlW
    else:
        wm = np.nan
    return wm

df = pd.read_csv('Chukchi17_19_ctd.csv')
wmS,wmB = [],[]
for index, row in df.iterrows():
    wmS.append(wmClass(row.T_surf, row.S_surf))
    wmB.append(wmClass(row.T_bot, row.S_bot))
df['Water_surf'] = wmS
df['Water_bot'] = wmB
df.to_csv('Chukchi17_19_ctd.csv')

In [ ]:
# Convert netCDFS to csvs, for Baker
import sys
sys.path.insert(1, 'C:/Users/Robert/Documents/projects/rltools')
#sys.path.insert(1, 'C:/Users/robert.levine/work/repositories/rltools')
from ctdTools import readCtd
files2017 = glob('ctdNC/*1701*')
files2019 = glob('ctdNC/*1901*')
[readCtd.nc2csv(file,2019) for file in files2019]

In [11]:
# Then stack all those csvs together by year
files = sorted(glob('ctdNC/*2017*.csv'))
bigDF = pd.concat([pd.read_csv(file) for file in files])
bigDF.to_csv('data_IES_CTD_02&PAR_2017.csv')

files = sorted(glob('ctdNC/*2019*.csv'))
bigDF = pd.concat([pd.read_csv(file) for file in files])
bigDF.to_csv('data_IES_CTD_02&PAR_2019.csv')

In [128]:
# MLD for 2012 and 2013
dfCasts = pd.concat([pd.read_csv('data_IES_CTD_02&PAR_2012.csv'), pd.read_csv('data_IES_CTD_02&PAR_2013.csv')])
dfMaster = pd.read_csv('Chukchi12_13_ctd.csv')
mld = []
for index, row in dfMaster.iterrows():
    dfCur = dfCasts[(dfCasts.Year == row.Year)&(dfCasts.StationNumber == row.Station)]
    try:
        mld1,mld2 = zmld_boyer(dfCur['PrimarySalinity PSU'].values, dfCur['PrimaryTemperature deg. C'].values,dfCur['Depth (m)'].values)
    except:
        print('Could not converge',row.Year, row.Station)
        mld2 = np.nan
    mld.append(mld2)
dfMaster['MLD'] = mld
dfMaster.to_csv('Chukchi12_13_ctd.csv')

Could not converge 2012 105.0
Could not converge 2012 101.0
Could not converge 2012 130.0
Could not converge 2013 29.0
Could not converge 2013 169.0
